In [1]:
import requests
import pandas as pd
import json
import time
from bs4 import BeautifulSoup
df = pd.read_html('1403.01.03.html')[0]

In [2]:
def download(img_url, name):
    with requests.Session() as session:
        data = session.get(img_url, stream=True, verify=False, headers= {'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'})
        with open(f"imgs/{name}.jpg", "wb") as f:
            f.write(data.content)
    print(f'{name} saved.')

In [3]:
def get_middle_info(base):
    # names
    fa_name = base['labels']['fa']['value']
    en_name = base['labels']['en']['value']

    # descriptions
    fa_desc = base['descriptions']['fa']['value']
    en_desc = base['descriptions']['en']['value']

    # wiki urls
    fa_url = base['sitelinks']['fawiki']['url']
    en_url = base['sitelinks']['enwiki']['url']

    # Gender
    gender = base['claims']['P21'][0]['mainsnak']['datavalue']['value']['id'].replace('Q6581097', 'man').replace('Q6581072', 'woman')

    # birth date
    birth_date = base['claims']['P569'][0]['mainsnak']['datavalue']['value']['time']

    # death date
    try:
        death_date = base['claims']['P70'][0]['mainsnak']['datavalue']['value']['time']
    except Exception as e:
        print(str(e))
        death_date = ''
    return fa_name, en_name, fa_desc, en_desc, fa_url, en_url, gender, birth_date, death_date

In [4]:
def get_img_and_caption(name, url):
    while True:
        try:
            wiki_page = requests.get(url, verify=False, headers= {'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}).text
            break
        except Exception as e:
            print(str(e))
            time.sleep(10)
    soup = BeautifulSoup(wiki_page)
    class_ = 'infobox-caption'
    #print(name, url)
    try:
        img_url = 'https:'+soup.find('td', class_='infobox-image').find('img')['src']
        print(img_url, '+'*50)
        download(img_url, name)
    except:
        print('I could not find or download image.')
    try:
        age_caption = soup.find("div", class_=class_).text
        print(age_caption, '************')
    except:
        age_caption = ''
        print('perhaps no age caption...')
    return age_caption

In [8]:
c = 19024

In [9]:
output = []

for i, url in enumerate(df['person']):
    # get wikidata page
    if i < c:
        continue
    print(i)
    while True:
        try:
            page = requests.get(url).text
            break
        except Exception as e:
            print(str(e))
            time.sleep(10)
    if 'Not Found' in page:
        continue
    soup = BeautifulSoup(page, 'html.parser')
    #         try:
    #             base = json.loads(soup.contents[0])['entities'][url.split('/')[-1]]
    #         except:
    #             base = json.loads(str(soup))['entities'][url.split('/')[-1]]
    #         print('base obtained.')
    try:
        base = json.loads(page)['entities'][url.split('/')[-1]]
    except:
        temp = json.loads(page)['entities']
        base = temp[list(temp.keys())[0]]
    try:
        fa_name, en_name, fa_desc, en_desc, fa_url, en_url, gender, birth_date, death_date = get_middle_info(base)
    except Exception as e:
        print(str(e), '......')
        continue
    #print('***', fa_name, en_name, fa_desc, en_desc, fa_url, en_url, gender, birth_date, death_date, '***')
    print('get wikipedia data')
    print('cap and img...')
    fa_cap = get_img_and_caption(fa_name, fa_url)
    time.sleep(60)
    en_cap = get_img_and_caption(en_name, en_url)

    output.append([fa_name, en_name, fa_desc, en_desc, fa_url, en_url, gender, birth_date, death_date, fa_cap, en_cap])
    time.sleep(90)
    c = i
    with open('res.txt', 'a+', encoding = 'utf-8') as file:
        file.write(',,, '.join([fa_name, en_name, fa_desc, en_desc, fa_url, en_url, gender, birth_date, death_date, fa_cap, en_cap]))
        file.write('\n')

19024
'fawiki' ......


In [7]:
base = json.loads(page)['entities'][list(base.keys())[0]]

KeyError: 'pageid'

In [ ]:
'Not Found' in page

In [ ]:
url.split('/')[-1]

In [ ]:
list(base.keys())[0]

In [ ]:
base

In [ ]:
url

In [ ]:
temp = json.loads(page)['entities']
base = temp[list(temp.keys())[0]]

In [ ]:
list(page.keys())[0]